In [1]:
import numpy as np
import nltk
# nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import porter
from nltk.text import TextCollection
import pandas as pd
import sys
import os
import json
import string
import re
from scipy import spatial

from data_loader import DataLoader

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fuissfcds/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [70]:
loader1 = DataLoader(DataLoader.data_path1)
table1 = loader1.load_table()

loader2 = DataLoader(DataLoader.data_path2)
table2 = loader2.load_table()
table = pd.concat([table1, table2], ignore_index = True)
table

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AIVZMEFEAGF3,B0090UTDKO,txlucky1,"[2, 2]",Although it would seem that most of the inform...,5,Surprisingly useful advice,1367280000,"04 30, 2013"
1,A17O3IALB0VVSA,B00ASFHQUY,mapleleafmanny,"[2, 2]",I've always loved the story of Peter Pan and I...,4,Better then the classic story,1379635200,"09 20, 2013"
2,A1Q6090OWANRMC,B00B6EFPEK,Bev Seabrook,"[18, 19]",Most people equate being frugal with being dep...,5,Start Living a Better Life Today,1359590400,"01 31, 2013"
3,A171L7UJTO7HXU,B00AQJRYWC,Ro13135,"[0, 0]","Good read, funny at times but author wrapped u...",4,Good Book but disapointing ending,1405728000,"07 19, 2014"
4,A2ISDBOVMRCRLO,B00ARY7ZUC,"C. L. Orlando ""always reading""","[0, 0]",I like the realistic viewpoints of the heroine...,4,Liking this series very much,1373846400,"07 15, 2013"
...,...,...,...,...,...,...,...,...,...
395,A3AL8GQ69QE7WN,B0014XCMVM,Jason Stein,"[2, 3]",Alanis Morissette always makes decent albums. ...,4,I have been missing the rapture this whole time.,1215734400,"07 11, 2008"
396,A1J5KCZC8CMW9I,B000B8QF00,"H3@+h ""Over 1500 reviews!""","[2, 3]","There are some who may still think of ""Jagged ...",4,"Almost ""Everything"" of hers.",1132012800,"11 15, 2005"
397,A1G7F9ZDJ07EMA,B00003NHAR,John,"[1, 2]",Well there is not much to say about an album t...,1,Volume 3,1069027200,"11 17, 2003"
398,AGZ28CC1MLUAD,B0000024JN,thaman,"[1, 3]","YEAH BRING BACK SOME GOOD TIMES,WHEN I WAS A L...",4,MY FIRST HIP HOP CASSET TAPE!,1133136000,"11 28, 2005"


# Collecting Documents

In [71]:
# Assume research can be performed on [reviewText] and [summary]
collection = []
size = len(table)

for i in range(size):
    collection.append(table.iloc[i]['reviewText'] + ' ' + table.iloc[i]['summary'])

# Preprocess

In [72]:
def preprocess(doc):
    # convert to lower cases
    doc = doc.lower()
    
    # remove stopwords
    stop_words = stopwords.words('english')
    word_tokens = word_tokenize(doc)
    doc = [w for w in word_tokens if w not in stop_words]
    
    # remove punctuations
    doc = [w for w in doc if w.isalnum()]
    
    # stem
    stemmer = porter.PorterStemmer()
    doc = [stemmer.stem(w) for w in doc]
    
    return doc

In [73]:
# preprocess all docs
for i in range(size):
    collection[i] = preprocess(collection[i])

# TF-IDF

In [74]:
# corpus
corpus = TextCollection(collection)

# tf-idf
vs = np.empty([len(corpus),size])
for i in range(size):
    for j, term in enumerate(corpus):
        vs[j][i] = corpus.tf_idf(term, collection[i])

# Query

In [95]:
query = input("Research for:")

Research for:musical innovation


In [96]:
qterm = preprocess(query)

qvector = np.zeros(len(corpus))

for i, term in enumerate(corpus):
        qvector[i] = corpus.tf_idf(term, qterm)
        


# Cosine Similarity

In [97]:
# Set N = 10 (Top N results)

result = np.zeros(size)

for i in range(size):
    result[i] = spatial.distance.cosine(vs[:,i], qvector)

# return the indices of smallest items
indices = np.argsort(result)[:10]
table.loc[indices,:]

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
318,A3FA5MCPWRL1H5,B00018D44U,Johncagebubblegum,"[7, 22]",Norah Jones basically embodies everything that...,1,Lethargic lullabies for a world gone absolutel...,1104364800,"12 30, 2004"
282,A2QFLRFUOIY270,B000002I9H,TDN,"[2, 2]",The first three Gabriel solo albums are incred...,4,Back On His Feet Again,1325462400,"01 2, 2012"
187,A8SEI0EJ2IQHL,B00I19Y70Q,Sara,"[1, 1]",What happens when people from different musica...,4,Collaboration by Michelle Lynn and Nevaeh Lee,1391644800,"02 6, 2014"
350,A1XVYE0A34BDKR,B0011HF6GE,Southern Man,"[0, 0]","I have a ""five listen rule"". I have to listen...",4,Sunday morning music,1349568000,"10 7, 2012"
245,A3NHJ4F4W1HLFX,B00000J7SR,Luppapa,"[3, 4]","First, don't expect a big monumental work from...",5,Funk at its best,1003190400,"10 16, 2001"
374,A14AWJWQL57ZKV,B000005HNA,dphilips2002,"[0, 0]",Start/stop. Soft/loud. Growl/whisper. These gu...,5,DICHOTOMY AS MUSIC = OPETH,1004918400,"11 5, 2001"
355,A3ITEDV7EYMF09,B00328G4XY,Drink Me,"[4, 4]","5 Stars for the music, which was brilliant whe...",3,It should have been a lot better,1269129600,"03 21, 2010"
236,A1D2C0WDCSHUWZ,B000MXPE74,"E. A Solinas ""ea_solinas""","[0, 0]",The Fratellis specialize in two things -- soli...,4,Second best killer I've ever seen,1173744000,"03 13, 2007"
372,A1D44YT0P8Q8XL,B00004TWRR,Rebecca*rhapsodyinblue*,"[0, 0]","""Even if they can't name him right away as the...",5,A Great Introduction To The Father of Bossa Nova,1269907200,"03 30, 2010"
274,A19MLEHXMXNVWK,B00005AV14,"Chris MB ""CMB""","[4, 4]",Elbow's effort on Asleep in the Back is truly ...,4,"Experimental, Satisfying Effort",1020988800,"05 10, 2002"


In [94]:
table.iloc[378]["reviewText"]


'This is a truly wonderful display of musical innovation & inspiration. Rickey Wright, in his so called review of this album, stated that it wasn\'t a "concept" album.  Nonsense!  Earth, Wind & Fire were a concept band in and of themselves, every one of their albums from the decade of the 70\'s had 2 main concepts, Oneness & African Pride!  They understood the rich and unprecedented history of the African people worldwide and brought this knowledge in mystical terms to the masses.Like all great prophets before, Maurice White in particular & the Elements as a whole, spoke in parables and those with eyes to see and ears to hear were amazed and electrified at what was being revealed!  Their lyrical content was second to none and greatly under appreciated by "so called critics" like Rickey Wright. This album is one of the finest expressions of musical & lyrical genius as you\'ll find."Serpentine Fire"- Pure power and mysticism, a song about the rising of the coiled, serpentine energy up th

In [4]:
#table.iloc[0]["summary"]


In [99]:
spatial.distance.cosine(vs[:,318], qvector)

0.28288600519271523

In [100]:
table.iloc[318]["reviewText"]


"Norah Jones basically embodies everything that is wrong with the world these days when it comes down to music. She proves that selling massive amounts of albums and getting loaded with all kinds of dubious music-awards really doesn't have anything to do with any form of musical ability, or creativity for that matter. Just like her first album, this one is also filled with drowsy, monotonous bores that should pass as 'jazzy' but calling this music jazz is a terrible insult to the genre that gave us Miles Davis, Wayne Shorter, John Coltrane, Herbie Hancock, Andrew Hill, McCoy Tyner, Sarah Vaughan, Ella Fitzgerald, Charles Mingus, and many more. 'That's not to say that this music is 'inoffensive'. For most people it IS inoffensive, just pleasant musical wallpaper that doesn't disturb a family dinner or an half an hour of washing the dishes, which declares the ridiculous amount of sales. For me, and thankfully many others who appreciate good music, this music IS offensive. It's mindnumbin